In [1]:
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2Model, BertTokenizer, BertModel, BertConfig
import torch
import torch.nn as nn
import os

In [2]:
import numpy as np

def save_to(base, x, y):
    np_input = x.detach().numpy()
    with open(os.path.join(base, 'dummy_in.dat'), 'wb') as f:
        np_input.tofile(f)

    np_output = y.detach().numpy()
    with open(os.path.join(base, 'dummy_out.dat'), 'wb') as f:
        np_output.tofile(f)

In [3]:
torch.manual_seed(2333)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [4]:
print(model)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwis

In [9]:
class GPT2_Component(nn.Module):
    def __init__(self, gpt2: GPT2Model):
        super().__init__()
        self.wte = gpt2.wte
        self.wpe = gpt2.wpe
        self.h0 = gpt2.h[0]
    
    def forward(self, src: Tensor) -> Tensor:
        

Sequential(
  (0): Embedding(50257, 768)
  (1): Embedding(1024, 768)
  (2): GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D()
      (c_proj): Conv1D()
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D()
      (c_proj): Conv1D()
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)


In [11]:
print(encoded_input)

{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [12]:
x = torch.randint(0, 50257, (64, 20))
x.dtype

torch.int64

In [13]:
y = gpt2_components(x)
y.size()

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [14]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = x = torch.randint(0, 50257, (64, 20))
torch.onnx.export(model,
                  dummy_input,
                  "./gpt2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [3]:
bert = BertModel.from_pretrained('bert-base-uncased')
bert.eval()
print(bert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
x = torch.randint(0, 30522, (64, 20))
y = bert(x)
y

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1550,  0.3241,  0.3431,  ..., -0.3762,  0.0519,  0.4771],
         [ 0.0592,  0.4553,  0.4656,  ..., -0.5866,  0.0515,  0.3473],
         [ 0.2676,  0.3613,  0.5149,  ..., -0.5446,  0.0296,  0.3255],
         ...,
         [-0.0035,  0.3264,  0.4768,  ..., -0.4506, -0.0524,  0.4459],
         [-0.0132,  0.2776,  0.2213,  ..., -0.3390, -0.1708,  0.2386],
         [ 0.0173,  0.2056,  0.4974,  ..., -0.5071, -0.0744,  0.6355]],

        [[ 0.1980,  0.0836,  0.2378,  ..., -0.1051,  0.0231, -0.0015],
         [ 0.2383,  0.1028,  0.2888,  ..., -0.0785,  0.0178, -0.2868],
         [ 0.2796,  0.1661,  0.3012,  ..., -0.1586,  0.0627, -0.2703],
         ...,
         [ 0.2763,  0.1047,  0.3133,  ..., -0.3115, -0.0179,  0.1814],
         [ 0.2490,  0.0797,  0.5146,  ..., -0.3036,  0.0258,  0.2452],
         [ 0.0299,  0.0828,  0.3787,  ..., -0.3461,  0.0578,  0.2345]],

        [[ 0.2631,  0.2989,  0.3120,  ..., -0.0099,  

In [5]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = torch.randint(0, 30522, (64, 20))
torch.onnx.export(bert,
                  dummy_input,
                  "./bert.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [10]:
save_to('.', x, y.last_hidden_state)